__for working on logging and status display tools__

In [ ]:
%cd ~/Dropbox/CanvasHacks

#Plotting 
# %matplotlib inline
# from matplotlib import pyplot as plt
# import seaborn as sns
# sns.set(style="whitegrid")

import pandas as pd

from CanvasHacks import environment


# This aren't used in the non-saving version
from CanvasHacks.FileTools import makeDataFileIterator

# Import the Canvas class
from canvasapi import Canvas
from canvasapi.quiz import QuizReport, Quiz
from canvasapi.requester import Requester
from canvasapi.conversation import Conversation

# Initialize a Canvas api objects
canvas = Canvas(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)

import inspect
def look_inside(obj):
    print(inspect.getmembers(obj, lambda a:not(inspect.isroutine(a))))

from CanvasHacks.PeerReviewed.Definitions import Review, InitialWork, MetaReview, Unit


from CanvasHacks.Repositories.quizzes import QuizRepository

from faker import Faker
fake = Faker()
from tests.factories.PeerReviewedFactories import discussion_entry_factory

In [ ]:
from CanvasHacks.Logging import *

# environment.CONFIG.set_test()


In [ ]:
StudentWorkLogger.write('taco 22')

In [ ]:

# def log_student_work(func):
#     # @wraps(func)
#     def wrapper(*args, **kwargs):
#         # handle logging
#         StudentWorkLogger.write("\n".join(args))
#         # call og function
#         func(*args, **kwargs)

#     return wrapper

@log_student_work
def test(content):
    print('yay', content)

In [ ]:
test('dog')

In [ ]:
j ="d"
j += 'i'
j

In [ ]:
class T:
    
    @property
    def j(self):
        return 'k'

T.j

# Message logging

In [ ]:
MessageLogger.write('hello')

In [ ]:
@log_message
def sender(to, fromf, p):
    print('j')
    return 'l'

In [ ]:
sender(to='f', fromf=2, p='s')

In [ ]:
@log_message
def esender(to, fromf, p):
    raise Exception('taco')

In [ ]:
esender('f', 2, 's')

In [ ]:
e =Exception('r')

In [ ]:
e.__str__()

In [ ]:
# machine readable notification logs

In [ ]:
message_types = [
    # reviewer gets initial work to review
    'initial-work',
    # author of original work gets feedback
    'peer-review-feedback',
    # reviewer gets metareview on feedback
    'metareview-feedback'
]

Want to log: 

review assignment id 

    Let's me tie back to the other student

receipient id

    The student who was sent the message
    
time sent

    duh
    
type of message

    - What the content was
    
    
So when I download new data I can lok up the review assignments by the assessor id from the new data, 